In [1]:
import cv2
from pyzbar.pyzbar import decode
import numpy as np

import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

import requests
import xml.etree.ElementTree as ET
from urllib.parse import urlparse, parse_qs


In [2]:
def leer_qr_dificil(ruta_imagen):
    # 1. Cargar la imagen
    img = cv2.imread(ruta_imagen)
    if img is None:
        print("Error: No se encontró la imagen.")
        return

    # Lista de intentos con diferentes procesamientos
    imagenes_a_probar = []
    
    # Intento A: Imagen original
    imagenes_a_probar.append(img)
    
    # Intento B: Escala de grises
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imagenes_a_probar.append(gray)
    
    # Intento C: Umbralización (Convertir a blanco y negro puro)
    # Esto ayuda mucho con fotos de pantallas para quitar el "gris" del fondo
    _, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
    imagenes_a_probar.append(thresh)

    # Intento D: Desenfoque Gaussiano + Umbralización (para quitar ruido de píxeles)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh_blur = cv2.threshold(blur, 100, 255, cv2.THRESH_BINARY)
    imagenes_a_probar.append(thresh_blur)

    print(f"Buscando QR en {ruta_imagen}...")

    # Probar cada método hasta encontrar algo
    for i, imagen_prueba in enumerate(imagenes_a_probar):
        codigos = decode(imagen_prueba)
        
        if codigos:
            print(f"\n¡Éxito en el intento {i+1}!")
            for codigo in codigos:
                # Los datos vienen en bytes, hay que decodificarlos a string
                datos = codigo.data.decode('utf-8')
                tipo = codigo.type
                print(f"Tipo: {tipo}")
                print(f"Datos: {datos}")
            return datos
            
    print("\nNo se pudo leer el QR. Intenta recortar la imagen solo al QR.")



In [3]:


def consultar_desde_url(url_qr):
    # 1. PARSEO DE LA URL
    # Extraemos los parámetros del link
    parsed_url = urlparse(url_qr)
    params = parse_qs(parsed_url.query)

    # Obtenemos los valores (parse_qs devuelve listas, tomamos el primer elemento)
    # Nota: Usamos .get() para evitar errores si falta algún dato
    uuid = params.get('id', [''])[0]
    rfc_emisor = params.get('re', [''])[0]
    rfc_receptor = params.get('rr', [''])[0]
    total_crudo = params.get('tt', ['0'])[0]
    sello = params.get('fe', [''])[0]

    # 2. LIMPIEZA DEL TOTAL
    # El link trae ceros extra (ej: 0000025652.130000) que a veces confunden al SOAP.
    # Lo convertimos a float y luego a string fijo con 2 o 6 decimales según prefieras.
    try:
        total_limpio = f"{float(total_crudo):.2f}" # Lo deja como "25652.13"
    except ValueError:
        total_limpio = total_crudo # Si falla, usamos el original

    print(f"Datos extraídos:")
    print(f" -> UUID: {uuid}")
    print(f" -> Emisor: {rfc_emisor}")
    print(f" -> Total detectado: {total_limpio}")
    print(rfc_receptor)

    # 3. CONSULTA SOAP (Igual que antes)
    url_sat = "https://consultaqr.facturaelectronica.sat.gob.mx/ConsultaCFDIService.svc"
    
    soap_request = f"""
    <soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:tem="http://tempuri.org/">
       <soapenv:Header/>
       <soapenv:Body>
          <tem:Consulta>
             <tem:expresionImpresa>?re={rfc_emisor}&amp;rr={rfc_receptor}&amp;tt={total_limpio}&amp;id={uuid}&amp;fe={sello}</tem:expresionImpresa>
          </tem:Consulta>
       </soapenv:Body>
    </soapenv:Envelope>
    """

    headers = {
        'Content-Type': 'text/xml; charset=utf-8',
        'SOAPAction': 'http://tempuri.org/IConsultaCFDIService/Consulta'
    }

    try:
        response = requests.post(url_sat, data=soap_request, headers=headers, timeout=10)
        
        if response.status_code == 200:
            root = ET.fromstring(response.text)
            datos = {}
            # Buscamos los tags clave
            for elemento in root.iter():
                if 'Estado' in elemento.tag:
                    datos['Estado'] = elemento.text
                elif 'EstatusCancelacion' in elemento.tag:
                    datos['EsCancelable'] = elemento.text
                elif 'ValidacionEFOS' in elemento.tag:
                    datos['EFOS'] = elemento.text
            return datos
        else:
            return {"Error": f"HTTP {response.status_code}"}
    except Exception as e:
        return {"Error": str(e)}

#A = leer_qr_dificil(r'c:\Users\ovill\OneDrive\Escritorio\ggg2.jpeg')


In [ ]:
url_usuario = A
resultado = consultar_desde_url(url_usuario)

print("\n--- RESULTADO FINAL ---")
print(resultado)